## Манифест

Мы занимаемся извлечением структурных элементов из сканированных документов.

Как правило, документы имеют логическую структуру: название, разбиение на главы, подглавы и т. д., нумерованные и маркированные списки.

Выделение логической структуры документа может пригодиться для автоматизированного анализа документов.
Мы хотим решать эту задачу как задачу классификации, нам нужно для каждой строки текста определить, к какому типу она относится.

На вход вам будут подаваться строки документов. Вам необходимо для каждой строки документа определить её тип. Выделяются следующие типы:

1)Заголовок

Название главы, секции, подглавы, параграфа - если название занимает несколько строк, остальные тоже относятся к названию. Заголовок визуально выделяется от основного текста отступом и/или шрифтом и/или жирностью. 

2)Список

Начало нумерованного или маркированного списка - если элемент списка занимает несколько строк - остальные строки не считаем списком.

3)Текст

Все остальное считается текстом.

Необходимо Заголовок пометить цифрой 1, Список - 2, Текст - 3

Примеры разметки:

Вход:

— — проверка действия автомата аварийного освещения - не реже одного раза в

Ответ: 2

Вход:

месяц;

Ответ: 3

Вход:

10.7.12 Система аварийного электроснабжения

Ответ: 2

Вход:

—  проведение регулярных осмотров оперативным персоналом находящегося

Ответ: 2

Вход:

А.5 Борная кислота

Ответ: 2

Вход:

Слив ядовитых и агрессивных жидкостей должен производить только

Ответ: 3

Вход:

® контроль за выполнением геодезических работ в процессе строительства и

Ответ: 2

Вход:

12 Меры пожарной безопасности при строительстве

Ответ: 1

Вход: 

основных зданий и сооружений

Ответ: 1


### JSON на выходе

[ {"type": '', "content": ''}, { }, { } ]

type - тип строчки - header - 1, list - 2, text - 3

content - содержимое строчки

### На будущее 

#### 1)Заголовки

Как правило, в начале документа идет глобальный заголовок - будем считать его заголовком 0 уровня.

Затем могут идти подзаголовки, начинающиеся со слов Глава, Раздел, Секция, Параграф, Статья + нумерация. Это заголовки n-го уровня, в зависимости от нумерации. (Раздел 1 - 1 уровень, Раздел/Подраздел 1.1 - 2 уровень и т. д.). Ограничимся тремя уровнями вложенности.

1)Заголовок (может быть в несколько строк)

2)Глава/параграф/секция/подзаголовок/раздел + нумерация

3)Подглава/подсекция/подпараграф/подраздел + нумерация

Могут быть подзаголовки без нумерации и без начальных слов, просто строки текста, выделенные курсивом, подчеркнуты, жирным шрифтом + отступ (???)

#### 2)Списки (уровень вложенности?)

Будем анализировать технические отчеты и статьи законов - считаем нумерованными списками следующее:

1)Вложенные списки с нумерацией 1.1, 1.1.1, и т.д.  - это списки 2, 3 уровня. Выделим такие списки как отдельный класс с неограниченным уровнем вложенности. Уровень таких списков определяется отдельно.

2)Списки + уровень вложенности (ограничимся третьим уровнем вложенности)

-Строки, начинающиеся с цифр (нумерованные списки) 1. 1) 1

-Списки, в которых нумерация ведется с помощью букв а), А), а, А, (a), (A)

-Списки маркированные точка, галка, кружок, тире, ромб, квадрат, крестик и т.д.

При этом можно учитывать отступ от начала строки и смотреть, продолжается ли нумерация далее
Если один список вложен в другой, то его уровень увеличивается

Заголовки и элементы списков(?) могут занимать несколько строк, это нужно также уметь как-то определять

#### 3)Просто строки

Что делать с английскими буквами? (или римскими цифрами, например)

Как извлекать информацию о шрифтах, отступах и т.д.?

Насколько сложная структура может быть (насколько сложные и разнообразные документы)? 

Как оценивать вместе с номером уровня заголовка?


## Извлечение текста из pdf
https://www.severcart.ru/blog/all/tesseract_ocr_python/

In [ ]:
from pdf2text import pdf2text

## Классификация заголовков

In [ ]:
import re
RE_LIST = re.compile(r'\d+(\.\d+)*\D') # для отдельного типа списка

In [ ]:
def list_proc(line):
    """
    если список, возвращает тип и глубину вложенности
    """
    match = RE_LIST.search(line)
    if match:
        if match.start() == 0:
            return 'list', match.string[match.span()[0]:match.span()[1]].count('.') + 1
    return

pipeline - классификатор, на входе - список строк текста, на выходе - тип строк

Извлекаем признаки:
смотрим на первое слово в каждой непустой строчке документа -> считаем слова

In [1]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
class string2features:
    def __init__(self):
        pass
    def fit(self):
        pass
    def predict(self):
        pass
    def fit_transform(self, X, y):
        return self.transform(X)
    def transform(self, X):
        """
        X - список строк
        """
        first_words = []
        for line in X:
            if line.split():
                first_words.append(line.split()[0])
            else:
                first_words.append('')
        return first_words

ppl = make_pipeline(string2features(), CountVectorizer(token_pattern=r'(?u)\b\w+\b'), LogisticRegression())

Обучаем модель:
логистическая регрессия, для каждой строки - тип строки + уровень???

как работать с уровнем? строка-название + строка-уровень

In [4]:
import json
with open("file_train.json", "r") as read_file:
    doc_with_labels = json.load(read_file)

In [5]:
y = [x["type"] for x in doc_with_labels]
X = [x["content"] for x in doc_with_labels]

Обучаем и сохраняем обученную модель

In [6]:
import pickle as pkl
clf = ppl.fit(X, y)
pkl.dump(clf, open("model.pkl", "wb"))

/home/nastyboget/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/nastyboget/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
